# Project 2, Part 5, Cleansing customer data

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering

Student: Sophie Yeh

Year: 2022

Semester: Spring

Section: 8

# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import csv
import numpy as np
import pandas as pd
import psycopg2
import math
import json
import pprint
from datetime import datetime as dt

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  

Remember you can freely use any code from the labs. You do not need to cite code from the labs.

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)


#connect to postgres database
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)
# create cursor for connections
cursor = connection.cursor()


# 2.5.1 Validate the city, state, and zip for stage_1_peak_customers against the zip_codes table

AGM does not want to give its customer list to 3rd party sales channels, including Peak Delivery.  For that reason, we can expect some variation in customer first and last names, and in the street.  However, the city, state, and zip should be validated by Peak's system, so we do not anticipate any issues.

Write a query that demonstrates that the city, state, and zip are valid for all records.  Like we did in 2.4, it's usually best to write a query that return errors.  In our case the query should not return anything.

Sort by stage_id

Pattern your code after the examples in the labs.  You may use as many code cells as you need.

In [15]:
query = """

select cu1.stage_id,
       cu1.city as peak_city,
       cu1.state as peak_state,
       cu1.zip as peak_zip,
       cu2.city as agm_city,
       cu2.state as agm_state,
       cu2.zip as agm_zip
from stage_1_peak_customers as cu1
     join zip_codes as cu2
         on cu1.zip = cu2.zip
where cu1.city != cu2.city or cu1.state != cu2.state or cu1.zip != cu2.zip
order by cu1.stage_id

;

"""
my_select_query_pandas(query, True, True)


,stage_id,peak_city,peak_state,peak_zip,agm_city,agm_state,agm_zip


# 2.5.2 Find all customer records in stage_1_peak_customers where any of first_name, last_name, and/or street do not match a customer in the customers table

AGM does not want to give its customer list to 3rd party sales channels, including Peak Delivery.  For that reason, we can expect some variation in customer first and last names, and in the street.  

Write a query that returns all customer records in state_1_peak_customers where any of the first_name, last_name, and/or street do not match a customer in the customers table

Sort by stage_id

Pattern your code after the examples in the labs.  You may use as many code cells as you need.

In [54]:
query = """


select cu1.stage_id,
       cu1.first_name as stage_first_name,
       cu1.last_name as stage_last_name,
       cu1.street as stage_street,
       cu2.first_name as customer_first_name,
       cu2.last_name as customer_last_name,
       cu2.street as customer_street
from stage_1_peak_customers as cu1
     left join customers as cu2
     on cu1.first_name = cu2.first_name and  cu1.last_name = cu2.last_name and  cu1.street = cu2.street
where cu2.first_name is null
order by cu1.stage_id
;

"""
my_select_query_pandas(query, True, True)


,stage_id,stage_first_name,stage_last_name,stage_street,customer_first_name,customer_last_name,customer_street
0,10,Hyrum,Knuckles,86668 Spenser Terrace,None,None,None
1,20,Roseann,Coyish,11707 American Ash Ter,None,None,None
2,24,Hali,Ducker,8 Orion Pass,None,None,None
3,26,Melantha,Golborn,6140 North Field Alley,None,None,None
4,36,Eleni,Jansen,66 Bartelt Hill,None,None,None
5,40,Clyve,Humonds,22 Brent Wood Hill,None,None,None
6,51,Rutledge,Hellwing,606 Gulf Plz,None,None,None
7,60,Kalli,Kemel,18373 Golf View Pass,None,None,None
8,72,Honina,Philson,28 Clarendon Plaza,None,None,None
9,73,Nicky,Haley,88424 Warrior Lane,None,None,None


# 2.5.3 Find the percentage of Peak's customer records that do not match to AGM's customers table

Write a query to find the percentage of Peak's customer records that do not match AGM's.  The percentage can be found by taking the number of customer records in stage_1_peak_customers that do not match and dividing by the number of customers records in stage_1_peak_customers and multiplying by 100.

Show the total number of Peak customer records in stage_1_peak_customers, the number that match to customers, the number that do not match to customers, and the percentage that do not match.

Show the percentage rounded to the nearest tenth.  It is not necessary to include a percent sign.

Pattern your code after the examples in the labs.  You may use as many code cells as you need.

In [65]:
query = """


select cu2.last_name is not null as has_match,
       count(cu1.stage_id),
       SUM(COUNT(cu1.stage_id)) OVER() AS total_count,
       cast(round(100*count(cu1.stage_id)/SUM(COUNT(cu1.stage_id)) OVER(),1) as varchar) as percentage
from stage_1_peak_customers as cu1
     left join customers as cu2
     on cu1.first_name = cu2.first_name and  cu1.last_name = cu2.last_name and  cu1.street = cu2.street
group by cu2.last_name is not null
;


"""
my_select_query_pandas(query, True, True).style.set_caption("Customer records with no AGM match")


,has_match,count,total_count,percentage
0,False,13,97,13.4
1,True,84,97,86.6


# 2.5.4 Executive summary on customer data

Write an executive summary on the customer data.  

The summary should be the equivalent to 3/4 to 1 page using standard fonts, spacing, and margins. 

As stated in the scenario, like most companies, AGM does not want to give out its customer list to 3rd party sales channels.  The downside is, as we have seen, that customer first names, last names, and street addresses will have some variations and not be exact matches.

The executives would like your recommendation of one of the following:
* Continue to withhold the customer data from 3rd party sales channels
* Give customer data to 3rd party sales channels

Recommend exactly one of these.

Support you recommendation with an explanation based on what you have seen from this preliminary data load.

You are not required to write any queries nor create any data visualizations.  However, you may want to include some to enhance and add quality to your submission.  Submissions with these tend to be higher quality, although, not always.

You may use any number of code cells and/or markdown cells. 

You may alternate between code cells and markdown cells.  That is perfectly fine.  It is understood that before we present it, an editor would pull out the text, results of queries, and data visualizations.

----
I recommend that we withhold customer data from 3rd party sales channels. Table 2.5.3 concluded that only 13.4% of Peak's customer records do not match to AGM's customers table. Of the 97 customers, it is only 13. Additionally, further investigation revealed that most mismatches can be easily resolved with common matching techniques. 

For all 13 rows that did not have any exact matches to AGM's customers table, soundex, levenshtein distance, metaphone and dmetaphone matching functions were tested. The levenshtein distance function was selected as the best joining technique because the query revealed that first and last names were mismatched due to spelling mistakes and the first syllable doesn’t always match. The streets names also differed in word spacing, capitalization, abbreviation, and typos. 
 
Matching all rows using the levenshtein distance required tuning the sensitivity for each column-pair. This can be impractical if the data contains a large number rows without exact matches. Since the sensitivity is set for an entire column-pair, it might not work on a dataset with a wide range of levenshtein distances from a single typo in a short name to multiple typos in a long name. However for this purpose where the dataset is small enough to be checked by eye, it is easy to manually tune the sensitivity settings. Furthermore, the problem of large datasets can be mitigated with using machine learning and deep learning to measure string distances. While it may take more functions to achieve accurate matches in large, diverse datasets, it would be feasible to do so.


In [103]:
query = """
select cu1.first_name,
       cu1.last_name,
       cu1.street,
       cu2.first_name as agm_first_name,
       cu2.last_name as agm_last_name,
       cu2.street as agm_street
from stage_1_peak_customers as cu1
     left join customers as cu2
     on levenshtein_less_equal(cu1.first_name, cu2.first_name, 5) < 6
        and levenshtein_less_equal(cu1.last_name, cu2.last_name, 5) < 3
         and levenshtein_less_equal(cu1.street, cu2.street, 5) < 6
where cu1.first_name != cu2.first_name or cu1.last_name != cu2.last_name or cu1.street != cu2.street
order by first_name
;
"""
my_select_query_pandas(query, True, True).style.set_caption("Customer records matching based on levenstein distance")
    


,first_name,last_name,street,agm_first_name,agm_last_name,agm_street
0,Clemens,McAloren,12532 Faragut Lane,Klemens,MacAloren,12532 Farragut Lane
1,Clyve,Humonds,22 Brent Wood Hill,Clyve,Hammonds,22 Brentwood Hill
2,Eleni,Jansen,66 Bartelt Hill,Eleni,Jansen,6 Bartelt Hill
3,Eula,Dewair,3 Crownhardt Road,Eula,Deware,3 Crownhardt Road
4,Hali,Ducker,8 Orion Pass,Halli,Ducker,8 Orin Pass
5,Honina,Philson,28 Clarendon Plaza,Junina,Philson,28 Clarendon Plaza
6,Hyrum,Knuckles,86668 Spenser Terrace,Hiram,Knuckles,86668 Spenser Terrace
7,I,Odunniom,4243 Sugar Park,Israel,O'Dunniom,4243 Sugar Park
8,Kalli,Kemel,18373 Golf View Pass,Kalli,Kemell,18373 Golf View Pass
9,Melantha,Golborn,6140 North Field Alley,Melantha,Golborn,6140 Northfield Alley


An alternative, straightforward option is to parse through the nulls through different variations of null matches, as shown in the 3 tables below. The downside to this is that it will not be able to extract all the nullmatches.

In [9]:
query = """


select cu1.first_name,
       cu1.last_name,
       cu1.street,
       cu2.first_name as agm_first_name,
       cu2.last_name as agm_last_name,
       cu2.street as agm_street
from stage_1_peak_customers as cu1
     left join customers as cu2
     on  cu1.last_name = cu2.last_name and  cu1.street = cu2.street
where cu1.first_name != cu2.first_name
order by first_name
;


"""
my_select_query_pandas(query, True, True).style.set_caption("Customer records with unmatched first name")

,first_name,last_name,street,agm_first_name,agm_last_name,agm_street
0,Honina,Philson,28 Clarendon Plaza,Junina,Philson,28 Clarendon Plaza
1,Hyrum,Knuckles,86668 Spenser Terrace,Hiram,Knuckles,86668 Spenser Terrace
2,Nicky,Haley,88424 Warrior Lane,Nickey,Haley,88424 Warrior Lane


In [27]:
query = """


select cu1.first_name,
       cu1.last_name,
       cu1.street,
       cu2.first_name as agm_first_name,
       cu2.last_name as agm_last_name,
       cu2.street as agm_street
from stage_1_peak_customers as cu1
     left join customers as cu2
     on cu1.first_name = cu2.first_name and  cu1.street = cu2.street
where cu1.last_name != cu2.last_name
order by first_name
;


"""
my_select_query_pandas(query, True, True).style.set_caption("Customer records with unmatched last name")

,first_name,last_name,street,agm_first_name,agm_last_name,agm_street
0,Eula,Dewair,3 Crownhardt Road,Eula,Deware,3 Crownhardt Road
1,Kalli,Kemel,18373 Golf View Pass,Kalli,Kemell,18373 Golf View Pass


In [16]:
query = """


select cu1.first_name,
       cu1.last_name,
       cu1.street,
       cu2.first_name as agm_first_name,
       cu2.last_name as agm_last_name,
       cu2.street as agm_street
from stage_1_peak_customers as cu1
     left join customers as cu2
     on cu1.first_name = cu2.first_name and  cu1.last_name = cu2.last_name
where cu1.street != cu2.street
order by first_name
;


"""
my_select_query_pandas(query, True, True).style.set_caption("Customer records with unmatched street")
    


,first_name,last_name,street,agm_first_name,agm_last_name,agm_street
0,Eleni,Jansen,66 Bartelt Hill,Eleni,Jansen,6 Bartelt Hill
1,Melantha,Golborn,6140 North Field Alley,Melantha,Golborn,6140 Northfield Alley
2,Roseann,Coyish,11707 American Ash Ter,Roseann,Coyish,11707 American Ash Terrace
3,Rutledge,Hellwing,606 Gulf Plz,Rutledge,Hellwing,606 Golf Plaza
